# Airdrops -- Exploratory Data Analysis

**[Johnnatan Messias](https://johnnatan-messias.github.io/), May 2025**.


In [1]:
# Standard Library Imports
import os
import gzip

# Third-Party Libraries
import polars as pl
from tqdm.notebook import tqdm

# Polars Configuration
pl.Config.set_float_precision(18)

polars.config.Config

In [2]:
import sys
code_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "src"))

sys.path.append(code_dir)

In [3]:
protocols = ["ens", "dydx", "1inch", "gemstone", "arbitrum",
             "optimism", "uniswap", "arkham", "lido", "tornado"]

In [4]:
data_dir = os.path.realpath(os.path.join(os.getcwd(), "..", "data"))

os.makedirs(data_dir, exist_ok=True)

In [5]:
# Loading dead/null addresses
dead_addresses = pl.read_csv(os.path.join(
    data_dir, "dead_addresses.csv"))['address']

In [6]:
decimals = 1e18

# Loading data


In [7]:
from pathlib import Path


def load_dataset(protocols):
    # Define base data directory
    base_path = Path(data_dir)

    # Load datasets using dictionary comprehension
    return {
        protocol: {
            'transfer': pl.scan_parquet(base_path / f"{protocol}_transfer.parquet"),
            'claim': pl.scan_parquet(base_path / f"{protocol}_claim.parquet")
        }
        for protocol in tqdm(protocols)
    }

In [8]:
# Load the dataset once
dfs = load_dataset(protocols)

# Define protocols with 'to' as receiver in claim
to_receiver_protocols = {"1inch", "uniswap", "lido", "tornado", "optimism"}

for protocol, data in dfs.items():
    print(protocol)

    # Rename for 'transfer'
    data['transfer'] = data['transfer'].rename(
        {"from": "sender", "to": "receiver"})

    # Determine claim renaming based on protocol
    claim_rename = {"to": "receiver"} if protocol in to_receiver_protocols else {
        "account": "receiver"}

    # Apply renaming for 'claim'
    data['claim'] = data['claim'].rename(claim_rename)

  0%|          | 0/10 [00:00<?, ?it/s]

ens
dydx
1inch
gemstone
arbitrum
optimism
uniswap
arkham
lido
tornado


In [9]:
# Loading exchange addresses
# Define data paths
cex_path = os.path.join(data_dir, "cex_addresses.csv")
dex_path = os.path.join(data_dir, "dex_addresses.csv")

# Load data
exchange_cex_addresses_df = pl.read_csv(cex_path)
exchange_dex_addresses_df = pl.read_csv(dex_path)

# Union set of the exchanges
exchange_addresses_set = set(exchange_cex_addresses_df['address'].to_list(
) + exchange_dex_addresses_df['address'].to_list())

# Output the number of addresses
print(
    f"There are {exchange_cex_addresses_df.shape[0]} CEX addresses and {exchange_dex_addresses_df.shape[0]} DEX addresses.")

print(f"There are {len(exchange_addresses_set)} exchange addresses in total.")

# Adding fake exchange created during the Gemstone airdrop attack.
exchange_addresses_set.add('0x7aa1ed8fb5e820e38c86cf3dd0a9eb0169a149ad')

There are 3169 CEX addresses and 685 DEX addresses.
There are 3569 exchange addresses in total.


# Tokens sold at exchanges


In [10]:
def initialize_account(balances_dict, address):
    if address not in balances_dict:
        balances_dict[address] = {
            'current': 0, 'history': [], 'n_sender': 0, 'n_receiver': 0
        }


def update_balance_history(balances_dict, address, block_number, timestamp, balance_before, balance_after):
    balances_dict[address]['history'].append({
        'block_number': block_number,
        'timestamp': timestamp,
        'balance_before': balance_before,
        'balance_after': balance_after
    })


def compute_account_balances(transfer_df):
    balances_history_dict = {}
    # Collecting data lazily
    data = transfer_df.select([
        pl.col("sender"),
        pl.col("receiver"),
        pl.col("amount"),
        pl.col("blockNumber"),
        pl.col("timestamp")
    ]).collect()
    for row in tqdm(data.iter_rows(named=True), desc='Loading balances', total=len(data)):
        sender, receiver = row['sender'], row['receiver']
        balance = float(row['amount'])
        block_number, timestamp = row['blockNumber'], row['timestamp']

        # Initialize sender and receiver accounts
        initialize_account(balances_history_dict, sender)
        initialize_account(balances_history_dict, receiver)

        # Update balance history
        sender_before = balances_history_dict[sender]['current']
        receiver_before = balances_history_dict[receiver]['current']
        update_balance_history(balances_history_dict, sender, block_number,
                               timestamp, sender_before, sender_before - balance)
        update_balance_history(balances_history_dict, receiver, block_number,
                               timestamp, receiver_before, receiver_before + balance)

        # Update transaction counts
        # update_transaction_counts(balances_history_dict, sender, receiver)
        balances_history_dict[sender]['n_sender'] += 1
        balances_history_dict[receiver]['n_receiver'] += 1

        # Update current balances
        balances_history_dict[sender]['current'] -= balance
        balances_history_dict[receiver]['current'] += balance

    print(f"Total unique addresses: {len(balances_history_dict)}")
    return balances_history_dict

In [11]:
balances = dict()
for protocol in protocols:
    print(protocol)
    balances[protocol] = compute_account_balances(dfs[protocol]['transfer'])

ens


Loading balances:   0%|          | 0/998619 [00:00<?, ?it/s]

Total unique addresses: 238936
dydx


Loading balances:   0%|          | 0/747372 [00:00<?, ?it/s]

Total unique addresses: 182159
1inch


...

Total unique addresses: 129695
lido


Loading balances:   0%|          | 0/1107586 [00:00<?, ?it/s]

Total unique addresses: 127786
tornado


Loading balances:   0%|          | 0/278455 [00:00<?, ?it/s]

Total unique addresses: 34968


In [12]:
exchange_balances = dict()
for protocol in tqdm(protocols, desc='Adding exchange balances'):
    balance = 0
    for exchange in exchange_addresses_set:
        if exchange in balances[protocol]:
            balance += balances[protocol][exchange]['current']
    exchange_balances[protocol] = balance
for exchange_balance in exchange_balances:
    print("The exchange balance for {} is {}".format(
        exchange_balance, exchange_balances[exchange_balance]))

Adding exchange balances:   0%|          | 0/10 [00:00<?, ?it/s]

The exchange balance for ens is 10710927.81554785
The exchange balance for dydx is 116277337.90959203
The exchange balance for 1inch is 557717062.2673184
The exchange balance for gemstone is 291997156478789.5
The exchange balance for arbitrum is 852070646.5485427
The exchange balance for optimism is 242895481.68695247
The exchange balance for uniswap is 79325697.70988528
The exchange balance for arkham is 143835350.313715
The exchange balance for lido is 155404705.45286748
The exchange balance for tornado is 669218.6397018757


In [26]:
for protocol in protocols:
    protocol_exchange_balance = exchange_balances[protocol]
    protocol_airdrop_supply = dfs[protocol]['claim'].select(
        pl.col('amount')).sum().collect().item()
    print(protocol, 100 *
          protocol_exchange_balance/protocol_airdrop_supply)

ens 54.55725354924504
dydx 69.41434878805683
1inch 666.7841259258996
gemstone 0.5794195062460137
arbitrum 77.97050511900203
optimism 146.10073897765383
uniswap 57.88485748986848
arkham 489.32713756466785
lido 3913.4462938739834
tornado 133.84372794037515


In [27]:
exchange_balances

{'ens': 10710927.81554785,
 'dydx': 116277337.90959203,
 '1inch': 557717062.2673184,
 'gemstone': 291997156478789.5,
 'arbitrum': 852070646.5485427,
 'optimism': 242895481.68695247,
 'uniswap': 79325697.70988528,
 'arkham': 143835350.313715,
 'lido': 155404705.45286748,
 'tornado': 669218.6397018757}

In [ ]:
# exhance_nodes = dict()
# for protocol in protocols:
#     exhance_nodes[protocol] = list()
#     for node in subgraphs[protocol].nodes():
#         if node in exchange_addresses_set:
#             exhance_nodes[protocol].append(node)
#     print(protocol, len(exhance_nodes[protocol]))

In [ ]:
# balances = dict()
# for protocol in tqdm(protocols):
#     amount = 0
#     for node in exchange_addresses_set:
#         edges = subgraphs[protocol].in_edges(node, data=True)
#         amount += sum([float(edge[2]['amount']) for edge in edges])
#     balances[protocol] = amount
# balances

In [ ]:
# General airdrop distribution statistics
for protocol in protocols:
    # Extract unique claimants using lazy operations
    claimant_df = dfs[protocol]['claim'].select(
        pl.col('receiver')).unique().collect()

    # Filter and aggregate data using lazy operations
    data = (
        dfs[protocol]['transfer']
        .filter(
            (pl.col('sender').is_in(claimant_df['receiver'])) &
            (pl.col('receiver').is_in(exchange_addresses_set))
        )
        .group_by('sender')
        .agg(pl.col('amount').sum().alias('total_sent'))
        .collect()  # Collect the aggregated data
    )

    # Calculate the balance directly from the aggregated data
    balance = data['total_sent'].sum()

    # Calculate the total claim amount using lazy evaluation
    total_claim_amount = dfs[protocol]['claim'].select(
        pl.col('amount').sum()).collect().item()

    # Calculate percentage balance
    percentage_balance = (balance / total_claim_amount) * 100

    # Output
    print(f"{protocol}: {percentage_balance:.2f}% | Balance: {balance} | Total Claimed: {total_claim_amount}")

ens: 83.07% | Balance: 16308526.164489243 | Total Claimed: 19632454.199476592
dydx: 41.40% | Balance: 69351276.63937142 | Total Claimed: 167511962.49729607
1inch: 553.95% | Balance: 463337538.94769806 | Total Claimed: 83642822.40415813
gemstone: 379.89% | Balance: 1.9144513323931466e+17 | Total Claimed: 5.0394775e+16
arbitrum: 130.23% | Balance: 1423180423.4554536 | Total Claimed: 1092811500.0
optimism: 190.65% | Balance: 316953675.8011883 | Total Claimed: 166252055.5245812
uniswap: 469.24% | Balance: 643052724.2253962 | Total Claimed: 137040499.276982
arkham: 22.83% | Balance: 6709524.230247119 | Total Claimed: 29394517.342645075
lido: 515.20% | Balance: 20458903.330653135 | Total Claimed: 3971044.797424059
tornado: 82.40% | Balance: 411992.5404550038 | Total Claimed: 500000.00000000006


In [ ]:
(
        dfs['uniswap']['transfer']
        .filter(
            (pl.col('sender').is_in(claimant_df['receiver'])) &
            (pl.col('receiver').is_in(exchange_addresses_set))
        )
        .sort(by=["blockNumber", "transactionIndex", "logIndex"])
        .group_by(["sender", "receiver"], maintain_order=True)
        .agg(pl.col("amount").first())
        .select(pl.col("amount").sum())
        .collect()  # Collect the aggregated data
    )

amount
f64
4.786738206636895426e6


In [119]:
def compute_distribution_stats(protocol, dfs, exchange_addresses_set):
    # Extract unique claimants using lazy operations
    claimant_df = dfs[protocol]['claim'].select('receiver').unique().collect()

    # Filter and aggregate data using lazy operations
    data = (
        dfs[protocol]['transfer']
        .filter(
            (pl.col('sender').is_in(claimant_df['receiver'])) &
            (pl.col('receiver').is_in(exchange_addresses_set))
        )
        .sort(by=['blockNumber', 'transactionIndex', 'logIndex'])
    )

    # Compute balance using only Polars operations
    balance = (
        data
        .select(['sender', 'amount'])
        .unique(subset=['sender'])  # Ensure each sender is counted once
        .select(pl.col('amount').cast(pl.Float64)).sum()
        .collect()
        .item()
    )

    # Calculate the total claim amount using lazy evaluation
    total_claim_amount = dfs[protocol]['claim'].select(pl.col('amount').sum()).collect().item()

    # Calculate percentage balance
    percentage_balance = (balance / total_claim_amount) * 100 if total_claim_amount != 0 else 0

    # Output
    print(f"{protocol}: {percentage_balance:.2f}% | Balance: {balance} | Total Claimed: {total_claim_amount}")



In [120]:
for protocol in protocols:
        compute_distribution_stats(protocol, dfs, exchange_addresses_set)

ens: 46.74% | Balance: 9175886.608028274 | Total Claimed: 19632454.199476592
dydx: 13.56% | Balance: 22708853.8403148 | Total Claimed: 167511962.49729607
1inch: 58.67% | Balance: 49077174.96056065 | Total Claimed: 83642822.40415813
gemstone: 95.01% | Balance: 4.788090624878638e+16 | Total Claimed: 5.0394775e+16
arbitrum: 28.64% | Balance: 312937356.2013145 | Total Claimed: 1092811500.0
optimism: 48.21% | Balance: 80151352.89150001 | Total Claimed: 166252055.5245812
uniswap: 43.69% | Balance: 59878693.30759989 | Total Claimed: 137040499.276982
arkham: 19.79% | Balance: 5818423.308377025 | Total Claimed: 29394517.342645075
lido: 65.75% | Balance: 2610834.1729569724 | Total Claimed: 3971044.797424059
tornado: 33.71% | Balance: 168571.8594565302 | Total Claimed: 500000.00000000006


In [121]:
# General airdrop distribution statistics
for protocol in protocols:
    # Extract unique claimants using lazy operations
    claimant_df = dfs[protocol]['claim'].select(
        pl.col('receiver')).unique().collect()

    # Filter and aggregate data using lazy operations
    data = (
        dfs[protocol]['transfer']
        .filter(
            (pl.col('sender').is_in(claimant_df['receiver'])) &
            (pl.col('receiver').is_in(exchange_addresses_set))
        )
        .sort(by=["blockNumber", "transactionIndex", "logIndex"])
        .collect()  # Collect the aggregated data
    )

    visited = set()
    balance = 0
    for row in data.iter_rows(named=True):
        if row['sender'] not in visited:
            balance += float(row['amount'])
            visited.add(row['sender'])

    # Calculate the total claim amount using lazy evaluation
    total_claim_amount = dfs[protocol]['claim'].select(
        pl.col('amount').sum()).collect().item()

    # Calculate percentage balance
    percentage_balance = (balance / total_claim_amount) * 100

    # Output
    print(f"{protocol}: {percentage_balance:.2f}% | Balance: {balance} | Total Claimed: {total_claim_amount}")

ens: 46.74% | Balance: 9175886.608028246 | Total Claimed: 19632454.199476592
dydx: 13.56% | Balance: 22708853.84031451 | Total Claimed: 167511962.49729607
1inch: 58.67% | Balance: 49077174.96056054 | Total Claimed: 83642822.40415813
gemstone: 95.01% | Balance: 4.788090624878644e+16 | Total Claimed: 5.0394775e+16
arbitrum: 28.64% | Balance: 312937356.2012874 | Total Claimed: 1092811500.0
optimism: 48.21% | Balance: 80151352.89152807 | Total Claimed: 166252055.5245812
uniswap: 43.69% | Balance: 59878693.307599925 | Total Claimed: 137040499.276982
arkham: 19.79% | Balance: 5818423.308377282 | Total Claimed: 29394517.342645075
lido: 65.75% | Balance: 2610834.17295697 | Total Claimed: 3971044.797424059
tornado: 33.71% | Balance: 168571.85945652984 | Total Claimed: 500000.00000000006


In [ ]:
ens: 46.74% | Balance: 9175886.608028274 | Total Claimed: 19632454.199476592
dydx: 13.56% | Balance: 22708853.8403148 | Total Claimed: 167511962.49729607
1inch: 58.67% | Balance: 49077174.96056065 | Total Claimed: 83642822.40415813
gemstone: 95.01% | Balance: 4.788090624878638e+16 | Total Claimed: 5.0394775e+16
arbitrum: 28.64% | Balance: 312937356.2013145 | Total Claimed: 1092811500.0
optimism: 48.21% | Balance: 80151352.89150001 | Total Claimed: 166252055.5245812
uniswap: 43.69% | Balance: 59878693.30759989 | Total Claimed: 137040499.276982
arkham: 19.79% | Balance: 5818423.308377025 | Total Claimed: 29394517.342645075
lido: 65.75% | Balance: 2610834.1729569724 | Total Claimed: 3971044.797424059
tornado: 33.71% | Balance: 168571.8594565302 | Total Claimed: 500000.00000000006